In [1]:
import os, sys
from pathlib import Path

sys.path.append('../src')
sys.path.append('../src/utils/')
from spiking_dataloader import WISDM_spiking_dataloader, WisdmDatasetParser
from inibitory_network.inibitory_network_h1_test import InibitoryLifNet
from output_process import OutputProcess

In [2]:
dataset = WisdmDatasetParser('../data/data_watch_subset_0_40.npz', norm=None, class_sublset='custom', subset_list=[0, 4, 6, 8, 9, 10, 14])

(6,)
(6,)
ytrain shape (55404, 18)
yval shape (18468, 18)
ytest shape (18469, 18)
num classes train dataset: 7 occurrences of each class:[3127 3044 3102 3047 3150 3087 2973]
num classes eval dataset: 7 occurrences of each class:[1035 1048 1122  996 1110 1053 1007]
num classes test dataset: 7 occurrences of each class:[1046 1048 1046 1036 1076 1026  982]


In [3]:
num_samples = 100
is_fixed = False

In [4]:
#val_set = dataset.get_validation_set(shuffle=False, subset=num_samples)
val_set = dataset.get_validation_set()
val_set[0].shape
num_samples = val_set[0].shape[0]

In [5]:
spiking_loader = WISDM_spiking_dataloader(val_set ,clear_intervall=40)

In [6]:
net = InibitoryLifNet(f"{Path.home()}/snntorch_network/notebook/Trained/network_best.npz", is_fixed=is_fixed)


leaky1 betas:0.6422713398933411
leaky1 vth:1.6034977436065674
leaky2 betas:0.1867157220840454
leaky2 vth:0.8977742195129395
recurrent betas:0.0
recurrent vth:0.7044668197631836
leaky3 betas:0.0
leaky3 vth:0.8803743720054626


In [7]:
net.print_weights()

###network weights###
linear1_w: [[ 0.01785181 -0.02856008  0.02428017 -0.5063322   0.25566658 -0.23295736]
 [-0.1067793   0.12840496 -0.0287918  -0.3225523   0.53797895 -0.43913406]
 [-0.07939682 -0.05350543  0.02370893  0.01309202  0.01555006 -0.02604755]
 ...
 [ 0.05031947  0.02002306  0.10233375  0.14626272  0.08159123  0.007313  ]
 [-0.10044242  0.29082403 -0.07642527  0.05289702  0.26825768 -0.43984282]
 [-0.0350589   0.11559165 -0.10274251 -0.32917562  0.32394004  0.28862447]]
linear2_w: [[ 0.18082134 -0.01966244  0.07137185 ... -0.04098366  0.06833572  0.02085055]
 [ 0.10025737 -0.08641492 -0.03791319 ... -0.01257913  0.02147631  0.05084423]
 [ 0.05788552 -0.08952484 -0.04185998 ...  0.02094941  0.0018082  -0.02334978]
 ...
 [-0.04137287  0.0017094   0.13737227 ...  0.08455733  0.03397838 -0.2366516 ]
 [ 0.1715753   0.08212121 -0.01064413 ...  0.0314768  -0.08438168 -0.09259132]
 [ 0.1911718  -0.09592617 -0.058932   ... -0.06258283 -0.12639815 -0.23494327]]
linear3_w: [[ 0.0924

In [8]:
out_sink = OutputProcess(7,num_samples,80, 0)

In [9]:
spiking_loader.data_out.connect(net.a_in)
net.s_out.connect(out_sink.spikes_in)
out_sink.label_in.connect_from(spiking_loader.label_out)

In [10]:

from lava.magma.core.run_conditions import RunSteps
from lava.magma.core.run_configs import Loihi1SimCfg, Loihi2SimCfg
import numpy as np
from tqdm import tqdm

for i in tqdm(range(num_samples)):
        out_sink.run(condition=RunSteps(num_steps=80),
                        run_cfg=Loihi1SimCfg(select_sub_proc_model=True,
                        select_tag='floating_pt'))
        net.reset_hidden_state()


ground_truth = val_set[1][:num_samples]
predictions = out_sink.pred_labels.get().astype(np.int32)

# Stop the execution
out_sink.stop()

accuracy = np.sum(ground_truth==predictions)/ground_truth.size * 100

print(f"\nGround truth: {ground_truth}\n"
      f"Predictions : {predictions}\n"
      f"Accuracy    : {accuracy}")


  0%|          | 0/7371 [00:00<?, ?it/s]

leaky1 dv:0.6422713398933411
leaky1 du:1.0
leaky1 vth:1.6034977436065674
leaky2 dv:0.1867157220840454
leaky2 du:1.0
leaky2 vth:0.8977742195129395
recurrent dv:0.0
recurrent du:1.0
leaky3 dv:0.0
leaky3 du:1.0
leaky3 vth:0.8803743720054626


/home/franzhd/lava/src/lava/magma/compiler/compiler_graphs.py:868: UserWarning: Cannot import module '<module 'dataloader' from '/home/franzhd/lava_AHPC/experiments/../src/utils/dataloader.py'>' when searching ProcessModels for Process 'OutputProcess'.
  warnings.warn(
/home/franzhd/lava/src/lava/magma/compiler/compiler_graphs.py:868: UserWarning: Cannot import module '<module 'dataloader' from '/home/franzhd/lava_AHPC/experiments/../src/utils/dataloader.py'>' when searching ProcessModels for Process 'WISDM_spiking_dataloader'.
  warnings.warn(
100%|██████████| 7371/7371 [14:54<00:00,  8.24it/s]


Ground truth: [2 5 4 ... 5 5 6]
Predictions : [2 5 4 ... 5 5 6]
Accuracy    : 87.87138787138787
